In [54]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
%run Functions.py

In [64]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [65]:
cat = []
for each in train.columns:
    if train[each].dtype == 'object':
        cat.append(each)

In [66]:
col = test.columns
label = train.y

In [67]:
import itertools
for f_1, f_2 in itertools.combinations(cat, 2):
    train[f_1+'_'+f_2] = train[f_1]+train[f_2]
    test[f_1+'_'+f_2] = test[f_1]+test[f_2]

In [68]:
interaction = [each for each in test.columns if each not in col]
interaction.append('ID')

In [69]:
from sklearn.preprocessing import LabelEncoder
for c in train.columns:
    if train[c].dtype == 'object':
        lbl = LabelEncoder()
        lbl.fit(list(train[c].values) + list(test[c].values))
        train[c] = lbl.transform(list(train[c].values))
        test[c] = lbl.transform(list(test[c].values))

In [70]:
train_ = train[col]
train_['y'] = label
test_ = test[col]
train_,test_ = get_additional_features(train_,test_)

In [71]:
interaction_data_train = train[interaction]
interaction_data_test = test[interaction]

In [72]:
train_ = train_.merge(interaction_data_train,on='ID',how='left')
test_ = test_.merge(interaction_data_test,on='ID',how='left')

## Same xgb, see the in-bag R2 and submission

In [73]:
y_train = train_['y'].values
y_mean = np.mean(y_train)

In [74]:
xgb_params = {
    'n_trees': 520, 
    'eta': 0.0045,
    'max_depth': 4,
    'subsample': 0.93,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'base_score': y_mean, # base prediction = mean(target)
    'silent': 1
}
# NOTE: Make sure that the class is labeled 'class' in the data file

dtrain = xgb.DMatrix(train_.drop('y', axis=1), y_train)
dtest = xgb.DMatrix(test_)

In [75]:
xgb_cvalid = xgb.cv(xgb_params, dtrain, num_boost_round=2000, early_stopping_rounds=20,
    verbose_eval=50, show_stdv=True,seed=42)
xgb_cvalid[['train-rmse-mean', 'test-rmse-mean']].plot()
print('Performance does not improve from '+str(len(xgb_cvalid))+' rounds')

[0]	train-rmse:12.6434+0.179136	test-rmse:12.6406+0.352672
[50]	train-rmse:11.2221+0.21883	test-rmse:11.2706+0.409113
[100]	train-rmse:10.1983+0.249851	test-rmse:10.31+0.460678
[150]	train-rmse:9.47225+0.275362	test-rmse:9.65252+0.503929
[200]	train-rmse:8.96341+0.294506	test-rmse:9.21214+0.54279
[250]	train-rmse:8.6072+0.308728	test-rmse:8.9227+0.573775
[300]	train-rmse:8.35821+0.319074	test-rmse:8.73444+0.599122
[350]	train-rmse:8.17246+0.324461	test-rmse:8.62072+0.618221
[400]	train-rmse:8.01312+0.316835	test-rmse:8.55208+0.632229
[450]	train-rmse:7.88217+0.309381	test-rmse:8.51+0.64496
[500]	train-rmse:7.7703+0.303667	test-rmse:8.48281+0.653668
[550]	train-rmse:7.66813+0.294556	test-rmse:8.4673+0.659792
[600]	train-rmse:7.56979+0.28123	test-rmse:8.45758+0.662973
[650]	train-rmse:7.47809+0.262059	test-rmse:8.45319+0.665341
[700]	train-rmse:7.39069+0.24672	test-rmse:8.45328+0.666564
Performance does not improve from 684 rounds


In [ ]:
[750]	train-rmse:7.37314+0.243044	test-rmse:8.43692+0.648636
Performance does not improve from 758 rounds


In [79]:
num_boost_rounds = 1250
# train model
model = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round=num_boost_rounds)
feature_import = get_feature_importance(model)

In [78]:
pred = model.predict(dtest)
sub = pd.DataFrame({'ID':test['ID'],'y':pred})
sub.to_csv('xgb_with_cat_interaction_interaction_not_in_decomp.csv',index=False)

In [80]:
r2_score(model.predict(dtrain),train.y)

0.27362487019282578

In [86]:
cc = feature_import[feature_import['Score']>1]

In [84]:
feature_import.shape

(160, 2)

## Add magic features with interaction

In [88]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [89]:
cat = []
for each in train.columns:
    if train[each].dtype == 'object':
        cat.append(each)
col = test.columns
label = train.y
import itertools
for f_1, f_2 in itertools.combinations(cat, 2):
    train[f_1+'_'+f_2] = train[f_1]+train[f_2]
    test[f_1+'_'+f_2] = test[f_1]+test[f_2]
interaction = [each for each in test.columns if each not in col]
interaction.append('ID')
from sklearn.preprocessing import LabelEncoder
for c in train.columns:
    if train[c].dtype == 'object':
        lbl = LabelEncoder()
        lbl.fit(list(train[c].values) + list(test[c].values))
        train[c] = lbl.transform(list(train[c].values))
        test[c] = lbl.transform(list(test[c].values))
train_ = train[col]
train_['y'] = label
test_ = test[col]
train_,test_ = get_additional_features(train_,test_,magic=True)
interaction_data_train = train[interaction]
interaction_data_test = test[interaction]
train_ = train_.merge(interaction_data_train,on='ID',how='left')
test_ = test_.merge(interaction_data_test,on='ID',how='left')

In [90]:
y_train = train_['y'].values
y_mean = np.mean(y_train)
xgb_params = {
    'n_trees': 520, 
    'eta': 0.0045,
    'max_depth': 4,
    'subsample': 0.93,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'base_score': y_mean, # base prediction = mean(target)
    'silent': 1
}
# NOTE: Make sure that the class is labeled 'class' in the data file

dtrain = xgb.DMatrix(train_.drop('y', axis=1), y_train)
dtest = xgb.DMatrix(test_)
xgb_cvalid = xgb.cv(xgb_params, dtrain, num_boost_round=2000, early_stopping_rounds=20,
    verbose_eval=50, show_stdv=True,seed=42)
xgb_cvalid[['train-rmse-mean', 'test-rmse-mean']].plot()
print('Performance does not improve from '+str(len(xgb_cvalid))+' rounds')

[0]	train-rmse:12.6432+0.179191	test-rmse:12.6405+0.352625
[50]	train-rmse:11.2062+0.214776	test-rmse:11.2714+0.406888
[100]	train-rmse:10.1657+0.24127	test-rmse:10.3081+0.459205
[150]	train-rmse:9.41013+0.252893	test-rmse:9.64641+0.506925
[200]	train-rmse:8.87086+0.256222	test-rmse:9.20212+0.547106
[250]	train-rmse:8.4783+0.249193	test-rmse:8.91182+0.577377
[300]	train-rmse:8.19271+0.243771	test-rmse:8.72723+0.598862
[350]	train-rmse:7.98521+0.238248	test-rmse:8.61083+0.615516
[400]	train-rmse:7.82443+0.236464	test-rmse:8.53854+0.629305
[450]	train-rmse:7.6839+0.232856	test-rmse:8.49387+0.639788
[500]	train-rmse:7.56056+0.223861	test-rmse:8.467+0.64509
[550]	train-rmse:7.45038+0.214731	test-rmse:8.45279+0.649665
[600]	train-rmse:7.35879+0.203955	test-rmse:8.44591+0.65256
Performance does not improve from 620 rounds


In [91]:
num_boost_rounds = 1250
# train model
model = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round=num_boost_rounds)
feature_import = get_feature_importance(model)

In [92]:
r2_score(model.predict(dtrain),train.y)

0.45929919565364163

In [93]:
pred = model.predict(dtest)
sub = pd.DataFrame({'ID':test['ID'],'y':pred})
sub.to_csv('xgb_with_cat_interaction_interaction_not_in_decomp_and_Magic_feature.csv',index=False)

## Only decom and magic features but many rounds

In [94]:
train = pd.read_csv('data/train_c.csv')
test = pd.read_csv('data/test_c.csv')

In [95]:
train,test = get_additional_features(train,test,magic=True)

In [96]:
col = test.columns

In [97]:
dtrain = xgb.DMatrix(train[col],train['y'])
dtest = xgb.DMatrix(test)

In [99]:
model = xgb.train(xgb_params,dtrain,num_boost_round=1250)

In [100]:
pred = model.predict(dtest)

In [102]:
r2_score(model.predict(dtrain),train.y)

0.46257635679369347

In [103]:
sub = pd.DataFrame({'ID':test['ID'],'y':pred})

In [104]:
sub.to_csv('xgb_1250Rounds_Alldecomp_magic.csv',index=False)